In [1]:
import numpy as np
#import tensorflow as tf

In [2]:
def init_transitions(a=0.2, b=0.3, g=0.2, l=1.1):
# transitionvectors 0-32
# stays, changes1. bit ..
    return np.array([g,a,b,b*l,b*(l**2),b*(l**3),b*(l**4),b*(l**5),b*(l**6),b*(l**7),b*(l**8),b*(l**9),b*(l**10),b*(l**11),b*(l**12),b*(l**13),b*(l**14),b*(l**15),b*(l**16),b*(l**17),b*(l**18),b*(l**19),b*(l**20),b*(l**21),b*(l**22),b*(l**23),b*(l**24),b*(l**25),b*(l**26),b*(l**27),b*(l**28),b*(l**29),b*(l**30),b*(l**31),b*(l**32) ])

print(init_transitions(l=1))

[0.2 0.2 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3
 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]


In [3]:
'''
    Function that helps making the output more readable
    
    Input: np.array - Subet in following repsentation: mask, bit1, bit2...
'''
def transform_rep_to_readableSubnet(r):
    ip_part_1 = np.packbits(r[1:9])
    ip_part_2 = np.packbits(r[9:17])
    ip_part_3 = np.packbits(r[17:25])
    ip_part_4 = np.packbits(r[25:])
    mask_part = r[0]
    return str(str(int(ip_part_1))+'.'+str(int(ip_part_2))+'.'+str(int(ip_part_3))+'.'+str(int(ip_part_4))+'/'+str(mask_part))
 

In [4]:
def markov_chain(alpha=0.2, beta=0.3, gamma=0.2, lambd=1.1, start="198.162.1.0/16", n = 1):
    # transitionvector
    tr = init_transitions(alpha,beta,gamma,lambd)
    # list of IPs 
    List_of_Subnets = []
    # parsing startIP
    startIP = np.asarray([int(x) for x in str(''.join([bin(int(x)+256)[3:] for x in start.split('/')[0]
.split('.')]))],dtype=int)    
    startM = int(start.split('/')[1])
    
    # np array representation of IP: Subn, bit1, bit2 ...
    s = np.hstack((np.asarray(startM,dtype=int),startIP))
    
    # markov chain
    i = 0  
    while i < n:
        List_of_Subnets.append(np.copy(s))
        # get transition vector for /n:
        tr_vec = tr[:s[0]+2]
        # L1 norm
        tr_vec = tr_vec/np.linalg.norm(tr_vec, ord=1)
        #print("transition probabilties: ",tr_vec)
        states = np.arange(s[0]+2) # subchange, changenothing, change1.bit, ...
        change =  np.random.choice(states,replace=True,p=tr_vec)
        # mask change
        if change == 0:
            s[0] = np.random.randint(32)
        # Change nothing
        elif change == 1:
            i += 1
        # flip the Bit
        else:
            s[change-1] = 1 - s[change-1]
            i += 1
        
    return List_of_Subnets    

In [5]:
import ipaddress

def format_ip(ip):
    if type(ip) != int:
        ip = ip.item()
    return ipaddress.IPv4Address(ip).exploded

stream = np.fromfile("/home/leon/data/caida_45s.bin", dtype=np.dtype(">u4"))

In [6]:
def check_for_hits(n=10000):
    a = markov_chain(n=n, gamma=0, start="111.37.227.254/32")

    hits = 0
    for i, f in enumerate(a):
        ip = np.frombuffer(np.packbits(f[1:]), dtype=np.dtype(">u4"))[0]

        # count = np.count_nonzero(stream == ip)
        if ip in stream:
            print(format_ip(ip), i)
            hits += 1

    print(hits, n)

In [7]:
from countminsketch import CountMinSketch as CMS
import importlib
import countminsketch
importlib.reload(countminsketch)


params = {
    "query_frequency": 1_000,
    "query_count": 1_000,
    "cms_width": 1000,
    "cms_depth": 10,
    "w1": 1,
    "w2": 0,
}

MASK_16 = 0xFFFF0000

sample = stream[:params["query_frequency"] * params["query_count"]]

gt_ips, gt_counts = np.unique(sample, return_counts=True)
order = np.argsort(gt_ips)
gt_ips = gt_ips[order]
gt_counts = gt_counts[order]


subnets, subnet_sizes = np.unique(gt_ips & MASK_16, return_counts=True)


order = np.argsort(subnet_sizes)[::-1]
subnets = subnets[order]
subnet_sizes = subnet_sizes[order]


queries = gt_ips[(gt_ips & MASK_16) == subnets[0]][:params["query_count"]]

#print(queries)

queries = sample[np.random.choice(params["query_frequency"] * params["query_count"], params["query_count"], replace=False)]

In [23]:
def increment_cms(cms, item, w=None):
    increment_by = 1

    if w is not None:
        if w == 0:
            return
        increment_by = np.random.choice(np.array([0, round(1/w)], dtype=np.int64), p=[1 - w, w])

    if increment_by > 0:
        cms.add(item, count=increment_by)
    
def evaluate_queries(w1=params["w1"], w2=params["w2"]):
    cms = CMS(params["cms_width"], params["cms_depth"])
    stream_pointer = 0
    
    error_absolute = np.empty(len(queries), dtype=np.int64)
    #error_relative = np.empty(len(queries), dtype=np.float64)
    
    mask = 0
    subnet = 0
    
    for i in range(params["query_count"]):

        for _ in range(params["query_frequency"]):
            ip = sample[stream_pointer]
            stream_pointer += 1

            if (ip & mask) == subnet:
                increment_cms(cms, ip, w=w1)
            else:
                increment_cms(cms, ip, w=w2)

        query = queries[i]
        mask = MASK_16
        subnet = query & mask
        
        count = np.count_nonzero(stream[:stream_pointer] == query)
        count_cms = cms.query(query)
        
        error_absolute[i] = count_cms - count
        #error_relative[i] = count_cms/max(count, 1) - 1
        
    return np.abs(error_absolute).mean()
    #print("Mean relative error:", f"{int(error_relative.mean() * 100)}%",)


w_all = np.linspace(0.7, 1, 50)
f = [139.396, 138.673, 138.289, 136.474, 136.884, 135.494, 135.088, 134.473, 134.05, 134.556, 133.481, 134.018, 133.355, 132.869, 133.947, 132.149, 131.922, 132.38, 132.774, 132.138, 132.508, 132.984, 131.927, 132.278, 132.383, 132.774, 132.671, 132.933, 132.646, 134.058, 134.249, 135.166, 135.614, 135.436, 137.091, 137.639, 138.849, 139.723, 140.594, 142.367, 143.413, 145.668, 146.857, 148.684, 151.18, 153.914, 157.044, 160.401, 163.71, 167.022]
for w in w_all:
    #f.append(evaluate_queries(w2=w))
    pass

print(f)
    
import matplotlib.pyplot as plt
%matplotlib

fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(w_all, f, "rx")
ax.plot(w_all, f)

#ax.set_title(f"MAE for different w2 values \n{params}")

ax.set_xlabel("$w_2$")
ax.set_ylabel("MAE")

plt.savefig("plot_w2_vs_mae.png", bbox_inches='tight', dpi=600)

#%time evaluate_queries(w2=1)

[139.396, 138.673, 138.289, 136.474, 136.884, 135.494, 135.088, 134.473, 134.05, 134.556, 133.481, 134.018, 133.355, 132.869, 133.947, 132.149, 131.922, 132.38, 132.774, 132.138, 132.508, 132.984, 131.927, 132.278, 132.383, 132.774, 132.671, 132.933, 132.646, 134.058, 134.249, 135.166, 135.614, 135.436, 137.091, 137.639, 138.849, 139.723, 140.594, 142.367, 143.413, 145.668, 146.857, 148.684, 151.18, 153.914, 157.044, 160.401, 163.71, 167.022]
Using matplotlib backend: Qt5Agg
